In [102]:
from tabulate import tabulate

f = open("input.txt", "r")

instructions = []
path = []

for line in f:
    [dir, n, color] = line.replace('\n', '').split(" ")
    instructions.append((dir, int(n), color.replace('(', '').replace(')', '')))

min_i, max_i, min_j, max_j = 0, 0, 0, 0

for (dir, n, color) in instructions:
    if len(path) > 0:
        (i, j, _) = path[-1]
    else:
        i, j = 0, 0
    match dir:
        case 'R':
            j += n
        case 'L':
            j -= n
        case 'U':
            i -= n
        case 'D':
            i += n
        case _:
            pass
    path.append((i, j, dir))
    min_i = min(min_i, i)
    min_j = min(min_j, j)
    max_i = max(max_i, i)
    max_j = max(max_j, j)

height = max_i - min_i + 1
width = max_j - min_j + 1
grid = [[0 for j in range(width)] for i in range(height)]
trenches = {}
rows_with_trenches = set()

prev_i, prev_j = 0, 0
path_index = 0
while path_index < len(path):
    (i, j, dir) = path[path_index]
    if path_index+1 < len(path):
        (_, _, next_dir) = path[path_index+1]
    else:
        (_, _, next_dir) = path[0]
    for ii in range(min(prev_i, i), max(prev_i, i)+1):
        first = min(prev_j, j)
        last = max(prev_j, j)
        for jj in range(first, last+1):
            grid[ii-min_i][jj-min_j] = 1
        if first == last:
            next_dir = dir
        # Remember which direction brought us to this trench, and which direction
        # we'll take when we leave it
        if ii-min_i in trenches:
            trenches[ii-min_i].add((first-min_j, last-min_j, dir, next_dir))
        else:
            trenches[ii-min_i] = {(first-min_j, last-min_j, dir, next_dir)}
        rows_with_trenches.add(ii-min_i)
    prev_i, prev_j = i, j
    path_index+=1

# Sort and merge trenches
for i in trenches:
    trenches[i] = sorted(list(trenches[i]))

total = 0
for trench in trenches.values():
    n = 0
    while n < len(trench)-1:
        (f, l, d, nd) = trench[n]
        (next_f, next_l, next_d, next_nd) = trench[n+1]
        if l == next_f or l == next_f-1:
            trench.pop(n)
            trench[n] = (f, next_l, d, next_nd)
        else:
            n+=1

rows_with_trenches = sorted(list(rows_with_trenches))

total = 0

row_index = 0
while row_index < len(rows_with_trenches):
    row = rows_with_trenches[row_index]
    trench = trenches[row]
    inside = False
    prev_dir, prev_next_dir = '', ''
    prev_l = 0
    i = 0
    while i < len(trench):
        (f, l, dir, next_dir) = trench[i]
        # Add the actual trench to the total
        total += l-f+1
        if dir == 'D' and prev_next_dir == 'U':
            # If we were going up after the previous trench, and down
            # when we got here, the gap between the trenches has been dug as well
            total += f-prev_l-1
        if i == len(trench)-1 and next_dir == 'U':
            # If we're at the last trench and we go up afterwards, then the gap
            # between this trench and the edge of our grid is inside the loop
            total += width - l
        prev_dir = dir
        prev_next_dir = next_dir
        prev_l = l
        i+=1
    row_index+=1

print(total)



40131


In [101]:
from queue import PriorityQueue
from heapq import heapify, heappush, heappop
from tabulate import tabulate

f = open("input.txt", "r")

instructions = []
path = []

for line in f:
    [dir, n, color] = line.replace('\n', '').split(" ")
    instructions.append((dir, int(n), color.replace('(', '').replace(')', '')))

actual_instructions = []

for (dir, n, color) in instructions:
    hex_distance = "0x" + color[1:6]
    distance = int(hex_distance, 16)
    hex_dir = color[6]
    match hex_dir:
        case '0':
            dir = 'R'
        case '1':
            dir = 'D'
        case '2':
            dir = 'L'
        case '3':
            dir = 'U'
    actual_instructions.append((dir, distance, ""))

instructions = actual_instructions

min_i, max_i, min_j, max_j = 0, 0, 0, 0

for (dir, n, color) in instructions:
    if len(path) > 0:
        (i, j) = path[-1]
    else:
        i, j = 0, 0
    match dir:
        case 'R':
            j += n
        case 'L':
            j -= n
        case 'U':
            i -= n
        case 'D':
            i += n
        case _:
            pass
    path.append((i, j))
    min_i = min(min_i, i)
    min_j = min(min_j, j)
    max_i = max(max_i, i)
    max_j = max(max_j, j)

height = max_i - min_i + 1
width = max_j - min_j + 1
trenches = {}
rows_with_trenches = set()

# Find all horizontal trenches, offset coordinates to start at (0,0)
prev_i, prev_j = 0, 0
path_index = 0
while path_index < len(path):
    (i, j) = path[path_index]
    if prev_i == i:
        first = min(prev_j, j)
        last = max(prev_j, j)
        if i-min_i in trenches:
            trenches[i-min_i].append((first-min_j, last-min_j))
        else:
            trenches[i-min_i] = [(first-min_j, last-min_j)]
        rows_with_trenches.add(i-min_i)
    prev_i, prev_j = i, j
    path_index+=1

# Sort and merge trenches
for i in trenches:
    trenches[i] = sorted(trenches[i])

def mergeTrenches(trench_list):
    trench_list.sort()
    n = 0
    while n < len(trench_list)-1:
        (f, l) = trench_list[n]
        (next_f, next_l) = trench_list[n+1]
        if l == next_f or l == next_f-1:
            trench_list.pop(n)
            trench_list[n] = (f, next_l)
        else:
            n+=1

for trench_list in trenches.values():
    mergeTrenches(trench_list)

rows_with_trenches = sorted(list(rows_with_trenches))

def addTrenchToList(trench, trench_list):
    (first, last) = trench
    to_add = last-first+1
    if len(trench_list) == 0:
        # No trench in list, adding trench
        trench_list.append(trench)
        return to_add
    first_trench_i = 0
    while first_trench_i < len(trench_list) and trench_list[first_trench_i][1] < first:
        first_trench_i+=1
    if first_trench_i == len(trench_list):
        # All trenches are before trench, no overlap
        trench_list.append(trench)
        return to_add
    first_trench = trench_list[first_trench_i]
    if first_trench[0] > last:
        # First trench is after trench, no overlap
        trench_list.insert(first_trench_i, trench)
        return to_add
    # Check if trench fits perfectly between two trenches
    if first_trench[1] == first:
        if first_trench_i+1 < len(trench_list):
            next_trench = trench_list[first_trench_i+1]
            if next_trench[0] == last:
                trench_list.insert(first_trench_i+1, trench)
                to_add = last-first-1
                return to_add
    to_add = 0
    last_trench_i = first_trench_i
    # Finding all trenches which overlap with trench
    while last_trench_i < len(trench_list) and trench_list[last_trench_i][0] <= last:
        last_trench_i+=1
    # For each overlapping trench, remove the overlap and keep the rest
    trench_index = first_trench_i
    while trench_index < min(len(trench_list), last_trench_i):
        (f, l) = trench_list[trench_index]
        # If we delete a trench, the index doesn't move but we end up with the next trench
        # anyway; which could be beyond trench
        if f > last:
            break
        deleted_trench = False
        if f < first:
            # Overlapping trench starts before trench
            truncated_trench = (f, first)
            trench_list[trench_index] = truncated_trench
            first = l
            trench_index+=1
        elif f == first:
            # Overlapping trench starts at the same position as trench
            del trench_list[trench_index]
            deleted_trench = True
        else:
            # Overlapping trench starts in the middle of trench
            truncated_trench = (first, f)
            trench_list[trench_index] = truncated_trench
            to_add += f-first
            first = l
            trench_index+=1
        if l <= last:
            # Last trench to overlap doesn't go beyond the end of trench
            first = l
        elif l > last:
            # Last trench to overlap goes beyond trench
            truncated_end_of_trench = (last, l)
            # Check if we made room in the list before
            if deleted_trench:
                trench_list.insert(trench_index, truncated_end_of_trench)
            else:    
                trench_list.insert(trench_index+1, truncated_end_of_trench)
            first = l
            break
    # The end of trench wasn't "used", it's a new trench
    if last > first:
        to_add += last-first
        trench_list.insert(trench_index, (first, last))

    return to_add
    

total = 0
row_index = 0
trenches_to_count = []
prev_row = 0
while row_index < len(rows_with_trenches):
    row = rows_with_trenches[row_index]
    row_total = 0
    for i in range(len(trenches_to_count)):
        (first, last) = trenches_to_count[i]
        row_total += last-first+1
    total += row_total*(row-prev_row-1)
    trenches_list = trenches[row]
    for trench in trenches_list:
        row_total += addTrenchToList(trench, trenches_to_count)
        mergeTrenches(trenches_to_count)
    total += row_total
    prev_row = row
    row_index+=1


print(total)

104454050898331
